## Clustering

In [1]:
import pandas as pd
import re
#from nltk.corpus import stopwords
import sklearn
#from sklearn.pipeline import Pipeline

#stopwords_rus = stopwords.words('russian')
from stop_words import get_stop_words
stopwords = get_stop_words('russian')
import numpy as np

In [2]:
from tqdm import tqdm

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

### Articles data

In [3]:
articles_df = pd.read_csv('articles_lemm_meta.tsv', sep = '\t', encoding = 'utf-8')

In [6]:
len(set(articles_df.final_rubrics))

19

In [7]:
articles_good_rubrics = articles_df[articles_df.final_rubrics != 'Мусор']
articles_good_rubrics = articles_good_rubrics.reset_index(drop = True)
articles_good_rubrics.shape

(24611, 13)

In [8]:
len(set(articles_good_rubrics.final_rubrics))

18

In [9]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import *
from sklearn.pipeline import *
from sklearn.preprocessing import Normalizer
from sklearn.metrics import *
from sklearn.cluster import *
import numpy as np

In [11]:
tfidf_vect = TfidfVectorizer(min_df = 2, stop_words = stopwords, max_features = 75000)#top 70% tfidf of vocabulary 
tfidf_vectorized = tfidf_vect.fit_transform(articles_good_rubrics.text)

## K-means

In [13]:
pipeline = Pipeline([
    #('vect', CountVectorizer(min_df = 2, stop_words = stopwords, max_features = 75000)),
    #('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 18, random_state = 42))
])


pipeline.fit(tfidf_vectorized)

Pipeline(memory=None,
     steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=1500, n_iter=5,
       random_state=None, tol=0.0)), ('clust', KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=18, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [14]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = articles_good_rubrics.final_rubrics

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.3452523978995787
Completeness: 0.36094858739839764
V-measure 0.3529260590458696
Adjusted Rand-Index: 0.12138728815660356


In [18]:
print(clust_labels)

[11  1  7 ...  1 12  7]


In [16]:
confusion_matrix(labels, clust_labels)

ValueError: Mix of label input types (string and number)

In [34]:
for i in range(18):
    print(i)
    print(labels.groupby(clust_labels).value_counts()[i])
    print('\n')

0
final_rubrics
Физика                 1014
Космос                  119
Технологии               79
Химия                    54
Биология                 16
Физиология человека      14
Футурология              13
Computer Science         10
Математика                5
История                   2
Науки о земле             2
Name: final_rubrics, dtype: int64


1
final_rubrics
История                1323
Биология                509
Науки о земле           113
Физиология человека     101
Культура                 69
Язык                     16
Технологии               12
Космос                   11
Computer Science          7
Социология                5
Физика                    5
Химия                     5
Математика                2
Футурология               1
Name: final_rubrics, dtype: int64


2
final_rubrics
Язык                   304
История                 19
Культура                18
Технологии               9
Computer Science         7
Физиология человека      7
Биология          

In [ ]:
sns.heatmap(data=confusion_matrix(labels, clust_labels), 
            annot=True, fmt="d", cbar=False)

#            , xticklabels=['male', 'female'], 
#                                             yticklabels=['male', 'female'])
plt.title("Confusion matrix")
plt.show()

## Hierarchical clustering

In [30]:
pipeline2 = Pipeline([
    #('vect', CountVectorizer(min_df = 2, stop_words = stopwords, max_features = 75000)),
    #('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', AgglomerativeClustering(n_clusters = 18, affinity = 'euclidean')) 
])

pipeline2.fit(tfidf_vectorized)

Pipeline(memory=None,
     steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=1500, n_iter=5,
       random_state=None, tol=0.0)), ('clust', AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
            connectivity=None, linkage='ward', memory=None, n_clusters=18,
            pooling_func=<function mean at 0x7ff32c908d08>))])

In [31]:
clust_labels2 = pipeline2.named_steps['clust'].labels_
#labels2 = articles_good_rubrics.final_rubrics

print("Homogeneity:", homogeneity_score(labels, clust_labels2))
print("Completeness:", completeness_score(labels, clust_labels2))
print("V-measure",  v_measure_score(labels, clust_labels2))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels2))

Homogeneity: 0.35088242047674745
Completeness: 0.35788221530484154
V-measure 0.35434775272969454
Adjusted Rand-Index: 0.18100658050869442


In [33]:
for i in range(18):
    print(i)
    print(labels.groupby(clust_labels2).value_counts()[i])
    print('\n')

0
final_rubrics
Физика                 490
Космос                  47
Технологии              40
Computer Science         6
Химия                    6
Физиология человека      5
Биология                 3
Математика               2
Футурология              2
Name: final_rubrics, dtype: int64


1
final_rubrics
Технологии          1022
Космос                64
Физика                10
История                5
Биология               3
Футурология            3
Computer Science       2
Науки о земле          2
Культура               1
Name: final_rubrics, dtype: int64


2
final_rubrics
Биология               2346
Физиология человека     928
Технологии              418
Психология              220
История                 176
Науки о земле           110
Культура                 72
Физика                   53
Социология               51
Химия                    50
Космос                   48
Computer Science         46
Язык                     35
Математика               23
Философия           

In [39]:
articles_good_rubrics.groupby(['final_rubrics']).size().sort_values(ascending = False)

final_rubrics
Технологии             5773
Космос                 4323
Биология               4049
История                2257
Физика                 1969
Физиология человека    1802
Культура                699
Науки о земле           575
Язык                    527
Химия                   506
Социология              464
Computer Science        421
Психология              380
Математика              249
Экономика               232
Футурология             188
Философия               142
Политология              55
dtype: int64

### Results

Общие (для двух методов):
1. Выделяется большой блок Технология (3000 текстов) + Космос, Физика 
2. Космос разбивается на несколько кластеров (1500, 800 + мелкие кластеры)
3. Выделяется блок гуманитарных наук (История, Социология, Культура, Экономика + Технологии)
4. Выделяется блок История + Биология
5. Выделяется блок Биология + Физиология
6. Выделяется блок Язык (300 текстов из 500)

K-means смешивает Технологии (3000) + Биология (2000) + Физика, Физиология, Космос <br>
Hierarchical clustering выдеялет 2 больших кластера Технологии + Космос+ Физика
